<a href="https://colab.research.google.com/github/lorenagueddes/DNC---Data-Cleaning-e-Data-Wrangling/blob/main/C%C3%B3pia_de_Exerc%C3%ADcio_Case_2_Gabarito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling - Case 2 

Nesse case, iremos continuar o case 1, partindo do ponto em que paramos e evoluindo com novas variáveis.

Se você fosse gerente em uma empresa, o que você faria diferente caso descobrisse que alguns de seus funcionários estão pensando em sair? Daria um aumento, buscaria contratar outro ou tentaria ver como melhorar? De toda forma, seria útil e evitaria muitos problemas!

No case de hoje, vamos fazer exatamente isso! 

Daily Happiness & Employee Turnover
Is There a Relationship Between Employee Happiness and Job Turnover?


Dataset: https://www.kaggle.com/harriken/employeeturnover?select=votes.csv


## Importe as bibliotecas necessárias:

In [ ]:
import pandas as pd
import numpy as np
# adicione outras aqui

## Importe os arquivos necessários:

- <b>Arquivo de churn Enriquecido (resultado da parte 1 com colunas adicionais)</b>:
Descreve se o funcionário ainda está na empresa, indicando os que já sairam (<i>churn</i>). O id do funcionário é único dentro da empresa somente (funcionários distintos em empresas diferentes podem ter o mesmo ID). Contem informações referentes às votações em que esse funcionário participou e aos comentários registrados por ele, além do genero da pessoa e do tipo de empresa.
Arquivo: data/churn.csv

- <b>Arquivo de Interação nos Comentários</b>:
Descreve os usuário que interagiram com os comentários postados, dando like ou deslike. Arquivo: data/commentInteractions.csv

- <b>Arquivo de Comentários Anônimos</b>:
Descreve cada comentário escrito pelos funcionários e uma visão geral da quantidade de likes e deslikes. Arquivo: data/comments_clean_anonimized.csv

In [ ]:
df_churn_enriq= pd.read_csv('data/df_churn_enriq.csv',sep=',')
df_comment_interactions= pd.read_csv('data/df_comment_interactions.csv',sep=',')
df_comments_clean_anonimized= pd.read_csv('data/df_comments_clean_anonimized.csv',sep=',')
df_churn_enriq.head()

,employee,companyAlias,numVotes,lastParticipationDate,stillExists,vote_mean,vote_mode,vote_min,vote_max,vote_count,qt_dias_diff_vote_min,qt_dias_diff_vote_max,qt_dias_diff_vote_median,likes_mean,likes_min,likes_max,dislikes_mean,dislikes_min,dislikes_max,qt_dias_diff_comment_min,qt_dias_diff_comment_max,qt_dias_diff_comment_median,commentId_count,companyType,gender
0,512,56aec740f1ef260003e307d6,4,2017-02-23 11:48:04,1,2.000000,1.0,1.0,3.0,3.0,2.0,37.0,31.0,7.500000,6.0,9.0,1.000000,0.0,2.0,2.0,30.0,16.0,2.0,Product,Male
1,2,56aec740f1ef260003e307d6,72,2017-03-17 00:00:00,1,2.239437,1.0,1.0,4.0,71.0,21.0,406.0,256.0,5.428571,0.0,12.0,4.785714,0.0,12.0,43.0,399.0,267.0,14.0,Product,Male
2,487,56aec740f1ef260003e307d6,14,2016-11-19 14:02:14,0,3.181818,3.0,2.0,4.0,22.0,2.0,66.0,26.5,6.000000,6.0,6.0,0.000000,0.0,0.0,22.0,22.0,22.0,1.0,Product,Female
3,3,56aec740f1ef260003e307d6,22,2017-02-16 00:00:00,1,3.476190,4.0,2.0,4.0,21.0,7.0,378.0,336.0,3.333333,0.0,14.0,0.888889,0.0,4.0,6.0,376.0,239.0,9.0,Product,Female
4,4,56aec740f1ef260003e307d6,195,2017-03-20 00:00:00,1,3.860825,4.0,1.0,4.0,194.0,1.0,368.0,105.5,7.500000,1.0,29.0,1.000000,0.0,4.0,33.0,271.0,107.0,10.0,Product,Female


## Enriquecimento dos dados

### Combinar 2 variáveis diferentes

a) Vamos combinar as colunas likes_mean e dislikes_mean calculando a razão de uma sobre a outra. Isso nos dará uma ideia da proporção entre likes e dislikes de um comentário.

In [ ]:
df_churn_enriq["rt_like_dislike"] = df_churn_enriq.likes_mean / df_churn_enriq.dislikes_mean

In [ ]:
df_churn_enriq.shape

(4064, 26)

b) Por ser uma divisão, essa variável ficou com alguns valores estranhos, como nulos e infinitos.
O nulo acontece quando dividimos 0 por 0. Já o infinito acontece quando temos algum like e 0 dislikes.
Calcule o valor de nulos e infinitos (np.inf).

In [ ]:
df_churn_enriq[df_churn_enriq["rt_like_dislike"] == np.inf].shape

(538, 26)

In [ ]:
df_churn_enriq.loc[df_churn_enriq["rt_like_dislike"].isna()].shape

(90, 26)

c) Vamos preencher os nulos com o valor 1 que indica mesma qt. de likes e dislikes

In [ ]:
df_churn_enriq.loc[df_churn_enriq["rt_like_dislike"].isna(),"rt_like_dislike"]=1

d) Plote o histograma dessa nova variável

In [ ]:
df_churn_enriq["rt_like_dislike"].describe()

count    4064.000000
mean             inf
std              NaN
min         0.000000
25%         5.466071
50%         5.611109
75%        11.167619
max              inf
Name: rt_like_dislike, dtype: float64

In [ ]:
# Using plotly.express
import plotly.express as px

fig = px.histogram(df_churn_enriq, x="rt_like_dislike")
fig.show()

e) Como essa variável tá com alguns picos indesejados e uma grande quantidade de outliers, faz sentido nesse caso aplicarmos a discretização por equal-frequncy usando 3 bins. 

Dica: para funcionar o algoritmo do KBins, antes substitua os infinitos por um valor alto

In [ ]:
df_churn_enriq.loc[df_churn_enriq["rt_like_dislike"] == np.inf,"rt_like_dislike"] = 50

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
# create the discretizer object with strategy uniform and 8 bins
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile') 

# fit the discretizer to the train set
discretizer.fit(df_churn_enriq[["rt_like_dislike"]].values)

# apply the discretisation
df_churn_enriq["rt_like_dislike"] = discretizer.transform(df_churn_enriq[["rt_like_dislike"]].values)

f) Verifique a nova distribuição

In [ ]:
df_churn_enriq["rt_like_dislike"].value_counts()

1.0    1673
2.0    1358
0.0    1033
Name: rt_like_dislike, dtype: int64

In [ ]:
# Using plotly.express
import plotly.express as px

fig = px.histogram(df_churn_enriq, x="rt_like_dislike")
fig.show()

### Criação de uma nova dimensão

Vamos retornar aos datasets de comentários! Já que não sabemos se os comentários são positivos ou não, poderiamos calcular uma dimensão de comentários que indique se o autor daquele comentário chegou a sair da empresa ou não. Isso pode nos indicar que um funcionário que dê like em algum comentário de alguém que saiu pode ser mais propenso a churn também. 

a) Faça um merge entre a df_comments_clean_anonimized e a df_churn_enriq, trazendo somente a variável stillExists.

In [ ]:
df_comments = df_comments_clean_anonimized.merge(df_churn_enriq[["employee","companyAlias","stillExists"]], on=["employee","companyAlias"])

In [ ]:
df_comments.head()

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate,stillExists
0,307,56aec740f1ef260003e307d6,58d018d7e010990004e38070,**********************************************...,4.0,0.0,2017-03-20 18:00:17,1
1,307,56aec740f1ef260003e307d6,58c913dfbd760e00043f1695,**********************************************...,8.0,1.0,2017-03-15 10:12:39,1
2,307,56aec740f1ef260003e307d6,58c736e732f72a00046f5614,********************************,5.0,0.0,2017-03-14 00:18:23,1
3,307,56aec740f1ef260003e307d6,58c53f19307b1e0004ad41af,**********************************************...,8.0,0.0,2017-03-12 12:28:34,1
4,307,56aec740f1ef260003e307d6,58c3ccf161ca670004c032fb,**********************,5.0,0.0,2017-03-11 10:09:36,1


b) Agora faça um merge entre a df_comment_interactions e essa nova base, trazendo somente a variável stillExists.

In [ ]:
df_comments_emp = df_comment_interactions.merge(df_comments[["commentId","stillExists"]], on=["commentId"])

In [ ]:
df_comments_emp.head()

,employee,companyAlias,liked,disliked,commentId,stillExists
0,307,56aec740f1ef260003e307d6,1,0,58d018d7e010990004e38070,1
1,36,56aec740f1ef260003e307d6,1,0,58d018d7e010990004e38070,1
2,276,56aec740f1ef260003e307d6,1,0,58d018d7e010990004e38070,1
3,24,56aec740f1ef260003e307d6,1,0,58d018d7e010990004e38070,1
4,382,56aec740f1ef260003e307d6,1,0,58d0179ae010990004e3806d,1


c) Nosso objetivo agora é resgatar o conceito de pivot table e calcular a media do stillExists para cada <employee, company> separando em liked e disliked comments.

In [ ]:
df_rate_liked_comments = pd.pivot_table(df_comments_emp, values='stillExists', index=['companyAlias', 'employee'],
                    columns=['liked','disliked'], aggfunc=np.mean, fill_value=0)

In [ ]:
df_rate_liked_comments.head()

liked                                     0         1
disliked                                  1         0
companyAlias             employee                    
5370af43e4b0cff95558c12a 18        0.444444  0.589912
                         20        0.000000  0.375000
                         21        0.125000  0.595238
                         22        0.500000  0.333333
                         23        0.500000  0.500000

In [ ]:
df_rate_liked_comments.columns = list(df_rate_liked_comments.columns.names)

In [ ]:
df_rate_liked_comments = df_rate_liked_comments.reset_index().rename(columns={'liked':'mean_stillExists_liked_comments',
                                            'disliked':'mean_stillExists_disliked_comments'})

In [ ]:
df_rate_liked_comments.head()

,companyAlias,employee,mean_stillExists_liked_comments,mean_stillExists_disliked_comments
0,5370af43e4b0cff95558c12a,18,0.444444,0.589912
1,5370af43e4b0cff95558c12a,20,0.000000,0.375000
2,5370af43e4b0cff95558c12a,21,0.125000,0.595238
3,5370af43e4b0cff95558c12a,22,0.500000,0.333333
4,5370af43e4b0cff95558c12a,23,0.500000,0.500000


d) Agora vamos fazer um merge com o dataset inicial!

In [ ]:
df_churn_enriq = df_churn_enriq.merge(df_rate_liked_comments, on=["employee","companyAlias"], how="left")

In [ ]:
df_churn_enriq.head()

,employee,companyAlias,numVotes,lastParticipationDate,stillExists,vote_mean,vote_mode,vote_min,vote_max,vote_count,qt_dias_diff_vote_min,qt_dias_diff_vote_max,qt_dias_diff_vote_median,likes_mean,likes_min,likes_max,dislikes_mean,dislikes_min,dislikes_max,qt_dias_diff_comment_min,qt_dias_diff_comment_max,qt_dias_diff_comment_median,commentId_count,companyType,gender,rt_like_dislike,mean_stillExists_liked_comments,mean_stillExists_disliked_comments
0,512,56aec740f1ef260003e307d6,4,2017-02-23 11:48:04,1,2.000000,1.0,1.0,3.0,3.0,2.0,37.0,31.0,7.500000,6.0,9.0,1.000000,0.0,2.0,2.0,30.0,16.0,2.0,Product,Male,2.0,0.0,1.000000
1,2,56aec740f1ef260003e307d6,72,2017-03-17 00:00:00,1,2.239437,1.0,1.0,4.0,71.0,21.0,406.0,256.0,5.428571,0.0,12.0,4.785714,0.0,12.0,43.0,399.0,267.0,14.0,Product,Male,0.0,1.0,0.979167
2,487,56aec740f1ef260003e307d6,14,2016-11-19 14:02:14,0,3.181818,3.0,2.0,4.0,22.0,2.0,66.0,26.5,6.000000,6.0,6.0,0.000000,0.0,0.0,22.0,22.0,22.0,1.0,Product,Female,2.0,0.0,1.000000
3,3,56aec740f1ef260003e307d6,22,2017-02-16 00:00:00,1,3.476190,4.0,2.0,4.0,21.0,7.0,378.0,336.0,3.333333,0.0,14.0,0.888889,0.0,4.0,6.0,376.0,239.0,9.0,Product,Female,0.0,1.0,1.000000
4,4,56aec740f1ef260003e307d6,195,2017-03-20 00:00:00,1,3.860825,4.0,1.0,4.0,194.0,1.0,368.0,105.5,7.500000,1.0,29.0,1.000000,0.0,4.0,33.0,271.0,107.0,10.0,Product,Female,2.0,1.0,0.983607


### Tratamentos em uma única variável

a) Plote os histogramas de cada variável numérica. As distribuicões parecem normais?

In [ ]:
bx_columns=['numVotes','qt_dias_diff_vote_max',
       'qt_dias_diff_vote_median', 'qt_dias_diff_comment_min']

In [ ]:
df_churn_enriq[bx_columns].describe()

,numVotes,qt_dias_diff_vote_max,qt_dias_diff_vote_median,qt_dias_diff_comment_min
count,4064.000000,4064.000000,4064.000000,4064.000000
mean,45.640994,148.462106,83.165108,33.290108
std,57.645434,125.680774,70.916590,46.409744
min,0.000000,1.000000,1.000000,1.000000
25%,6.000000,45.000000,26.000000,10.000000
50%,20.000000,130.000000,72.500000,17.000000
75%,65.000000,207.000000,117.125000,28.000000
max,273.000000,563.000000,336.000000,215.000000


In [ ]:
# Using plotly.express
import plotly.express as px

fig = px.histogram(df_churn_enriq, x="qt_dias_diff_vote_median")
fig.show()

b) Muitas variáveis do dataset estão com uma cauda muito longa na distribuição, vamos melhorar isso?

Aplique o Box-Cox Transformation para as variáveis a seguir:

In [ ]:
df_churn_enriq['numVotes_pos']=df_churn_enriq['numVotes']+1

In [ ]:
from sklearn.preprocessing import PowerTransformer

# create the power transformer object with method 'box-cox'
bct_numVotes = PowerTransformer(method='box-cox', standardize=False)
bct_qt_dias_diff_vote_max = PowerTransformer(method='box-cox', standardize=False)
bct_qt_dias_diff_vote_median = PowerTransformer(method='box-cox', standardize=False)
bct_qt_dias_diff_comment_min = PowerTransformer(method='box-cox', standardize=False)

# apply the transformation to your data
df_churn_enriq['numVotes_box-cox'] = bct_numVotes.fit_transform(df_churn_enriq[['numVotes_pos']])
df_churn_enriq['qt_dias_diff_vote_max_box-cox'] = bct_qt_dias_diff_vote_max.fit_transform(df_churn_enriq[['qt_dias_diff_vote_max']])
df_churn_enriq['qt_dias_diff_vote_median_box-cox'] = bct_qt_dias_diff_vote_median.fit_transform(df_churn_enriq[['qt_dias_diff_vote_median']])
df_churn_enriq['qt_dias_diff_comment_min_box-cox'] = bct_qt_dias_diff_comment_min.fit_transform(df_churn_enriq[['qt_dias_diff_comment_min']])

In [ ]:
# Lembre de dropar essa coluna:
df_churn_enriq =df_churn_enriq.drop('numVotes_pos',axis=1)

d) Visualize novamente após a transformação

In [ ]:
# Using plotly.express
import plotly.express as px

fig = px.histogram(df_churn_enriq, x="qt_dias_diff_vote_median_box-cox")
fig.show()

### Variáveis Categóricas

a) Faça um value_counts() das variáveis categóricas

In [ ]:
categ_cols = ["gender", "companyType"]
for col in categ_cols:
    print(df_churn_enriq[col].value_counts(dropna=False))

Female    2491
Male      1573
Name: gender, dtype: int64
Service    3018
Product    1046
Name: companyType, dtype: int64


b) Aplique o One-Hot Encoding para cada uma delas, aplicando o drop_first.

In [ ]:
df_churn_enriq[["gender_Male","companyType_Service"]] = pd.get_dummies(df_churn_enriq[categ_cols], prefix=categ_cols, drop_first=True)

In [ ]:
df_churn_enriq.head()

,employee,companyAlias,numVotes,lastParticipationDate,stillExists,vote_mean,vote_mode,vote_min,vote_max,vote_count,qt_dias_diff_vote_min,qt_dias_diff_vote_max,qt_dias_diff_vote_median,likes_mean,likes_min,likes_max,dislikes_mean,dislikes_min,dislikes_max,qt_dias_diff_comment_min,qt_dias_diff_comment_max,qt_dias_diff_comment_median,commentId_count,companyType,gender,rt_like_dislike,mean_stillExists_liked_comments,mean_stillExists_disliked_comments,numVotes_box-cox,qt_dias_diff_vote_max_box-cox,qt_dias_diff_vote_median_box-cox,qt_dias_diff_comment_min_box-cox,gender_Male,companyType_Service
0,512,56aec740f1ef260003e307d6,4,2017-02-23 11:48:04,1,2.000000,1.0,1.0,3.0,3.0,2.0,37.0,31.0,7.500000,6.0,9.0,1.000000,0.0,2.0,2.0,30.0,16.0,2.0,Product,Male,2.0,0.0,1.000000,1.655730,7.822055,6.833475,0.704922,1,0
1,2,56aec740f1ef260003e307d6,72,2017-03-17 00:00:00,1,2.239437,1.0,1.0,4.0,71.0,21.0,406.0,256.0,5.428571,0.0,12.0,4.785714,0.0,12.0,43.0,399.0,267.0,14.0,Product,Male,0.0,1.0,0.979167,4.630097,23.573827,17.893925,4.125844,1,0
2,487,56aec740f1ef260003e307d6,14,2016-11-19 14:02:14,0,3.181818,3.0,2.0,4.0,22.0,2.0,66.0,26.5,6.000000,6.0,6.0,0.000000,0.0,0.0,22.0,22.0,22.0,1.0,Product,Female,2.0,0.0,1.000000,2.840827,10.418957,6.302382,3.334595,0,0
3,3,56aec740f1ef260003e307d6,22,2017-02-16 00:00:00,1,3.476190,4.0,2.0,4.0,21.0,7.0,378.0,336.0,3.333333,0.0,14.0,0.888889,0.0,4.0,6.0,376.0,239.0,9.0,Product,Female,0.0,1.0,1.000000,3.314402,22.864652,20.038600,1.871862,0,0
4,4,56aec740f1ef260003e307d6,195,2017-03-20 00:00:00,1,3.860825,4.0,1.0,4.0,194.0,1.0,368.0,105.5,7.500000,1.0,29.0,1.000000,0.0,4.0,33.0,271.0,107.0,10.0,Product,Female,2.0,1.0,0.983607,5.798258,22.603556,12.207508,3.810250,0,0


## Validação & Limpeza Final

a) Ver correlação entre variáveis e variável target

In [ ]:
for col in df_churn_enriq.select_dtypes(include=np.number).columns:
    print(col, df_churn_enriq[col].corr(df_churn_enriq['stillExists']))

employee -0.13061930593362
numVotes 0.08261231402977397
stillExists 1.0
vote_mean 0.036602443752447804
vote_mode 0.024327597708567218
vote_min -0.019976048116231295
vote_max 0.04955892091679806
vote_count 0.05169395874536424
qt_dias_diff_vote_min 0.01774534408085189
qt_dias_diff_vote_max 0.04458513068028731
qt_dias_diff_vote_median 0.07389656088690945
likes_mean -0.00467164206035979
likes_min 0.08659450789337489
likes_max 0.023660705460176075
dislikes_mean -0.04865795353413274
dislikes_min 0.06342970972491727
dislikes_max 0.03957882960319471
qt_dias_diff_comment_min 0.07984348456655167
qt_dias_diff_comment_max -0.014643999691361547
qt_dias_diff_comment_median 0.011666737355124851
commentId_count 0.038574974786206494
rt_like_dislike 0.04631619592154073
mean_stillExists_liked_comments 0.0739605285694057
mean_stillExists_disliked_comments 0.3379784293482078
numVotes_box-cox 0.10911112163094176
qt_dias_diff_vote_max_box-cox 0.08378458353986956
qt_dias_diff_vote_median_box-cox 0.09927372374

b) Verifique de novo as distribuições e os tipos das variáveis restantes

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
df_churn_enriq.describe()

,employee,numVotes,stillExists,vote_mean,vote_mode,vote_min,vote_max,vote_count,qt_dias_diff_vote_min,qt_dias_diff_vote_max,qt_dias_diff_vote_median,likes_mean,likes_min,likes_max,dislikes_mean,dislikes_min,dislikes_max,qt_dias_diff_comment_min,qt_dias_diff_comment_max,qt_dias_diff_comment_median,commentId_count,rt_like_dislike,mean_stillExists_liked_comments,mean_stillExists_disliked_comments,numVotes_box-cox,qt_dias_diff_vote_max_box-cox,qt_dias_diff_vote_median_box-cox,qt_dias_diff_comment_min_box-cox,gender_Male,companyType_Service
count,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,3007.000000,3007.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000
mean,192.518455,45.640994,0.852608,2.949092,2.981053,1.859744,3.765748,45.355315,9.157972,148.462106,83.165108,6.621822,1.675689,13.477362,1.135128,0.103346,3.330217,33.290108,145.930856,102.120325,8.565207,1.079970,0.657053,0.916686,3.222967,13.403065,9.647534,3.038490,0.387057,0.742618
std,206.118146,57.645434,0.354540,0.596316,0.789037,0.869260,0.485830,58.683164,17.706919,125.680774,70.916590,4.231692,2.515155,9.533752,1.048029,0.458624,3.942949,46.409744,93.598221,70.456009,18.528229,0.762945,0.445815,0.197129,1.549058,6.282314,4.563803,1.406228,0.487137,0.437245
min,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,6.000000,1.000000,2.705882,3.000000,1.000000,4.000000,5.000000,1.000000,45.000000,26.000000,4.000000,0.000000,8.000000,0.333333,0.000000,1.000000,10.000000,97.750000,64.000000,0.000000,0.000000,0.000000,0.953718,2.013851,8.636581,6.239919,2.435986,0.000000,0.000000
50%,131.000000,20.000000,1.000000,3.000000,3.000000,2.000000,4.000000,19.000000,3.000000,130.000000,72.500000,6.771111,1.000000,12.000000,1.206733,0.000000,2.000000,17.000000,139.000000,94.000000,2.000000,1.000000,1.000000,1.000000,3.213016,14.294602,10.300969,3.036959,0.000000,1.000000
75%,244.000000,65.000000,1.000000,3.266667,3.000000,2.000000,4.000000,65.000000,7.000000,207.000000,117.125000,6.771111,1.000000,15.000000,1.206733,0.000000,3.000000,28.000000,165.000000,116.125000,9.000000,2.000000,1.000000,1.000000,4.513130,17.602663,12.786742,3.616377,1.000000,1.000000
max,999.000000,273.000000,1.000000,4.000000,4.000000,4.000000,4.000000,323.000000,100.000000,563.000000,336.000000,20.500000,12.000000,44.000000,5.600000,3.000000,17.000000,215.000000,468.000000,351.000000,279.000000,2.000000,1.000000,1.000000,6.203779,27.078685,20.038600,6.134444,1.000000,1.000000


c) Há ainda algum valor nulo? Se sim, trate-os.

In [ ]:
mean_cols=['mean_stillExists_liked_comments','mean_stillExists_disliked_comments']
for col in mean_cols:
    avg=df_churn_enriq[col].mean()
    df_churn_enriq.loc[df_churn_enriq[col].isna(),col]=avg
    print(col,"- substituição de nulos por",avg)

mean_stillExists_liked_comments - substituição de nulos por 0.6570527436627515
mean_stillExists_disliked_comments - substituição de nulos por 0.9166864577580646


In [ ]:
df_churn_enriq.dtypes

employee                                int64
companyAlias                           object
numVotes                                int64
lastParticipationDate                  object
stillExists                             int64
vote_mean                             float64
vote_mode                             float64
vote_min                              float64
vote_max                              float64
vote_count                            float64
qt_dias_diff_vote_min                 float64
qt_dias_diff_vote_max                 float64
qt_dias_diff_vote_median              float64
likes_mean                            float64
likes_min                             float64
likes_max                             float64
dislikes_mean                         float64
dislikes_min                          float64
dislikes_max                          float64
qt_dias_diff_comment_min              float64
qt_dias_diff_comment_max              float64
qt_dias_diff_comment_median       

d) Vamos ver quais colunas manter

In [ ]:
list(df_churn_enriq)

['employee',
 'companyAlias',
 'numVotes',
 'lastParticipationDate',
 'stillExists',
 'vote_mean',
 'vote_mode',
 'vote_min',
 'vote_max',
 'vote_count',
 'qt_dias_diff_vote_min',
 'qt_dias_diff_vote_max',
 'qt_dias_diff_vote_median',
 'likes_mean',
 'likes_min',
 'likes_max',
 'dislikes_mean',
 'dislikes_min',
 'dislikes_max',
 'qt_dias_diff_comment_min',
 'qt_dias_diff_comment_max',
 'qt_dias_diff_comment_median',
 'commentId_count',
 'companyType',
 'gender',
 'rt_like_dislike',
 'mean_stillExists_liked_comments',
 'mean_stillExists_disliked_comments',
 'numVotes_box-cox',
 'qt_dias_diff_vote_max_box-cox',
 'qt_dias_diff_vote_median_box-cox',
 'qt_dias_diff_comment_min_box-cox',
 'gender_Male',
 'companyType_Service']

In [ ]:
df_churn_final=df_churn_enriq[['employee',
 'companyAlias',
 #'numVotes',
 'vote_mean',
 'vote_mode',
 'vote_min',
 'vote_max',
 'vote_count',
 'qt_dias_diff_vote_min',
 #'qt_dias_diff_vote_max',
 #'qt_dias_diff_vote_median',
 'likes_mean',
 'likes_min',
 'likes_max',
 'dislikes_mean',
 'dislikes_min',
 'dislikes_max',
 #'qt_dias_diff_comment_min',
 'qt_dias_diff_comment_max',
 'qt_dias_diff_comment_median',
 'commentId_count',
 #'companyType',
 #'gender',
 'rt_like_dislike',
 'mean_stillExists_liked_comments',
 'mean_stillExists_disliked_comments',
 'numVotes_box-cox',
 'qt_dias_diff_vote_max_box-cox',
 'qt_dias_diff_vote_median_box-cox',
 'qt_dias_diff_comment_min_box-cox',
 'gender_Male',
 'companyType_Service',
 # previsão
 'lastParticipationDate',
 'stillExists']]          

In [ ]:
df_churn_final.head()

,employee,companyAlias,vote_mean,vote_mode,vote_min,vote_max,vote_count,qt_dias_diff_vote_min,likes_mean,likes_min,likes_max,dislikes_mean,dislikes_min,dislikes_max,qt_dias_diff_comment_max,qt_dias_diff_comment_median,commentId_count,rt_like_dislike,mean_stillExists_liked_comments,mean_stillExists_disliked_comments,numVotes_box-cox,qt_dias_diff_vote_max_box-cox,qt_dias_diff_vote_median_box-cox,qt_dias_diff_comment_min_box-cox,gender_Male,companyType_Service,lastParticipationDate,stillExists
0,512,56aec740f1ef260003e307d6,2.000000,1.0,1.0,3.0,3.0,2.0,7.500000,6.0,9.0,1.000000,0.0,2.0,30.0,16.0,2.0,2.0,0.0,1.000000,1.655730,7.822055,6.833475,0.704922,1,0,2017-02-23 11:48:04,1
1,2,56aec740f1ef260003e307d6,2.239437,1.0,1.0,4.0,71.0,21.0,5.428571,0.0,12.0,4.785714,0.0,12.0,399.0,267.0,14.0,0.0,1.0,0.979167,4.630097,23.573827,17.893925,4.125844,1,0,2017-03-17 00:00:00,1
2,487,56aec740f1ef260003e307d6,3.181818,3.0,2.0,4.0,22.0,2.0,6.000000,6.0,6.0,0.000000,0.0,0.0,22.0,22.0,1.0,2.0,0.0,1.000000,2.840827,10.418957,6.302382,3.334595,0,0,2016-11-19 14:02:14,0
3,3,56aec740f1ef260003e307d6,3.476190,4.0,2.0,4.0,21.0,7.0,3.333333,0.0,14.0,0.888889,0.0,4.0,376.0,239.0,9.0,0.0,1.0,1.000000,3.314402,22.864652,20.038600,1.871862,0,0,2017-02-16 00:00:00,1
4,4,56aec740f1ef260003e307d6,3.860825,4.0,1.0,4.0,194.0,1.0,7.500000,1.0,29.0,1.000000,0.0,4.0,271.0,107.0,10.0,2.0,1.0,0.983607,5.798258,22.603556,12.207508,3.810250,0,0,2017-03-20 00:00:00,1


## Fim!

Não se esqueça de salvar o dataset final!

In [ ]:
df_churn_final.to_csv('data/df_churn_final.csv',sep=',',index=False)